In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from itertools import zip_longest
import time,random , os
import pandas as pd
from urllib.parse import urljoin

In [2]:
def build_search_url(city: str) -> str:
    # Dates factices -> souvent plus stable. Adapte si tu veux.
    return (
        "https://www.booking.com/searchresults.html"
        f"?ss={city}&lang=fr&checkin=2025-11-10&checkout=2025-11-12&group_adults=2"
    )



In [3]:
MAX_PROPS_PER_CITY = 30      # pour tester: commence à 10 ou 20
MAX_REVIEW_PAGES   = 30      # nb max de pages d'avis par établissement
DATA_DIR           = "data"

In [4]:
def make_driver(headless=False):
    from selenium.webdriver.chrome.options import Options
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1280,900")
    options.add_argument("--lang=fr-FR")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    driver = webdriver.Chrome(options=options)
    driver.set_page_load_timeout(60)
    driver.implicitly_wait(2)  # petit implicite pour réduire le code
    return driver

In [5]:
def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

In [6]:
def sleep(a=2.0, b=4.0):
    time.sleep(random.uniform(a, b))

In [7]:
def discover_properties_for_city(driver, city, limit=MAX_PROPS_PER_CITY):
    url = build_search_url(city)
    driver.get(url)
    sleep()
    if city=="Laayoune":
        limit=28
    print(limit)
    props, seen = [], set()
    while len(props) < limit:
        # cartes d'établissement
        cards = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="property-card"]')
        for c in cards:
            try:
                a = c.find_element(By.CSS_SELECTOR, 'a[data-testid="title-link"]')
                href = a.get_attribute("href") or ""
                name = a.text.strip()
                full = urljoin("https://www.booking.com", href.split("?")[0])
                #print(f"name: {name} link: {full}")
                if full and full not in seen:
                    seen.add(full)
                    props.append({"city": city, "name": name, "url": full})
                    if len(props) >= limit:
                        break
            except:
                continue
        if len(props) >= limit:
            break

        # pagination listing
        # next_btn = None
        # for sel in [
        #     'button[aria-label*="Page suivante"]',
        #     'a[aria-label*="Page suivante"]',
        #     'button[aria-label*="Next"]',
        #     'a[aria-label*="Next"]'
        # ]:
        #     try:
        #         cand = driver.find_element(By.CSS_SELECTOR, sel)
        #         if cand and cand.is_enabled():
        #             next_btn = cand
        #             break
        #     except:
        #         continue
        # if not next_btn:
        #     break
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(0.5, 1.0)
        print(len(props))
        #next_btn.click()
        #sleep()
    driver.quit()
    return props


In [32]:
#os.makedirs("data/")

In [8]:
CITIES = [
    "Laayoune","Dakhla"
]
for city in CITIES:
    driver = make_driver()
    print(f"scraping {city}")
    L=discover_properties_for_city(driver,city)
    save_to_csv(L,"data/"f"{city}.csv")

scraping Laayoune
28
25
25
scraping Dakhla
30
25
25


In [ ]:
# IN_DIR="data"
# files=os.listdir(IN_DIR)
# files
# df=pd.read_csv("places_Marrakech.csv")
# for f in files:
#     dff=pd.read_csv(f"data/{f}")
#     df=pd.concat([df,dff],ignore_index=True)
# df.to_csv("data/all_places.csv")

In [24]:
out=pd.read_csv("all_places.csv")
out=out.drop(["Unnamed: 0"],axis=1)
out.to_csv("all_places.csv",index=False)

In [ ]:
data=pd.read_csv("all_places.csv")

,city,name,url
0,Marrakech,Riad L'EncensOriental\nUne nouvelle fenêtre va...,https://www.booking.com/hotel/ma/riad-encens.f...
1,Marrakech,Wazo Hotel\nUne nouvelle fenêtre va s'ouvrir,https://www.booking.com/hotel/ma/wazo.fr.html
2,Marrakech,L'Orientale By La Siredrah\nUne nouvelle fenêt...,https://www.booking.com/hotel/ma/orientale-by-...
3,Marrakech,Wazo Appart-Hôtel\nUne nouvelle fenêtre va s'o...,https://www.booking.com/hotel/ma/wazo-appart.f...
4,Marrakech,Hotel Central Palace\nUne nouvelle fenêtre va ...,https://www.booking.com/hotel/ma/central-palac...
...,...,...,...
503,Tetouan,Appartement avec vue panoramique\nUne nouvelle...,https://www.booking.com/hotel/ma/appartement-a...
504,Tetouan,VILLA MATI Maison D'hôtes\nUne nouvelle fenêtr...,https://www.booking.com/hotel/ma/villa-mati-ma...
505,Tetouan,Dar Tetuania\nUne nouvelle fenêtre va s'ouvrir,https://www.booking.com/hotel/ma/riad-al-balad...
506,Tetouan,Hôtel Atenas\nUne nouvelle fenêtre va s'ouvrir,https://www.booking.com/hotel/ma/atenas.fr.html
